In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler


In [2]:
# Read the dataset
data = pd.read_csv('output_fivedirections.csv')

# Encode categorical columns using LabelEncoder
categorical_columns = ['Source UUID', 'Source type', 'Destination UUID', 'Destination type', 'Edge type']
encoder = LabelEncoder()

for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

# Split the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the target variable (Label)
y = LabelEncoder().fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Create a neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [4]:
# Compile the model with a learning rate of 0.001
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-6)
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr])


Epoch 1/30
110841/110841 [==============================] - 270s 2ms/step - loss: 0.1443 - accuracy: 0.9425 - val_loss: 0.0922 - val_accuracy: 0.9654
Epoch 2/30
110841/110841 [==============================] - 270s 2ms/step - loss: 0.1074 - accuracy: 0.9598 - val_loss: 0.0859 - val_accuracy: 0.9700
Epoch 3/30
110841/110841 [==============================] - 265s 2ms/step - loss: 0.1001 - accuracy: 0.9631 - val_loss: 0.0804 - val_accuracy: 0.9713
Epoch 4/30
110841/110841 [==============================] - 270s 2ms/step - loss: 0.0960 - accuracy: 0.9649 - val_loss: 0.0801 - val_accuracy: 0.9716
Epoch 5/30
110841/110841 [==============================] - 260s 2ms/step - loss: 0.0932 - accuracy: 0.9661 - val_loss: 0.0754 - val_accuracy: 0.9731
Epoch 6/30
110841/110841 [==============================] - 258s 2ms/step - loss: 0.0913 - accuracy: 0.9668 - val_loss: 0.0742 - val_accuracy: 0.9732
Epoch 7/30
110841/110841 [==============================] - 270s 2ms/step - loss: 0.0895 - accuracy:

In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Decode the predictions to 'Attack' or 'Normal'
y_pred = LabelEncoder().fit(['Attack', 'Normal']).inverse_transform(y_pred.ravel())

In [ ]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))